**IMPORTING NECESSARY PACKAGES AND MODULES**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import word_tokenize
import numpy

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**DATA PREPARING** - copy pasting the exact data from google 
classroom assignment pdf.

In [ ]:
columns = ['Words in Doc', 'class']
rows = []

rows = [['Kyoto Osaka Taiwan', 'yes'], 
        ['Japan Kyoto', 'yes'],
        ['Taipei Taiwan', 'no'],
        ['Macao Taiwan Shanghai', 'no'],
        ['London', 'no']
        ]

training_data = pd.DataFrame(rows, columns=columns)
training_data

,Words in Doc,class
0,Kyoto Osaka Taiwan,yes
1,Japan Kyoto,yes
2,Taipei Taiwan,no
3,Macao Taiwan Shanghai,no
4,London,no


CLASS PROBABILITIES   | **PRIOR PROBABILITIES**


In [ ]:
yes_count = 0 
no_count = 0 
for q in range(len(rows)):
  if rows[q][1] == 'yes':
    yes_count=yes_count+1
  if rows[q][1] == 'no':
    no_count=no_count+1

class_yes = yes_count/len(rows)
class_no = no_count/len(rows)
class_yes , class_no

(0.4, 0.6)

**TERM DOCUMENT MATRIX FOR CLASS = "YES"**

****

In [ ]:
yes_docs = [row['Words in Doc'] for index,row in training_data.iterrows() if row['class'] == 'yes']
vec_yes = CountVectorizer()
X_yes = vec_yes.fit_transform(yes_docs)
tdm_yes = pd.DataFrame(X_yes.toarray(), columns=vec_yes.get_feature_names())
tdm_yes


,japan,kyoto,osaka,taiwan
0,0,1,1,1
1,1,1,0,0


**TERM DOCUMENT MATRIX FOR CLASS = "NO"**

In [ ]:
no_docs = [row['Words in Doc'] for index,row in training_data.iterrows() if row['class'] == 'no']
vec_no = CountVectorizer()
X_no = vec_no.fit_transform(no_docs)
tdm_no = pd.DataFrame(X_no.toarray(), columns=vec_no.get_feature_names())
tdm_no


,london,macao,shanghai,taipei,taiwan
0,0,0,0,1,1
1,0,1,1,0,1
2,1,0,0,0,0


**WORD COUNT FOR CLASS = YES**

In [ ]:
word_list_yes = vec_yes.get_feature_names();    
count_list_yes = X_yes.toarray().sum(axis=0) 
freq_yes = dict(zip(word_list_yes,count_list_yes))
freq_yes

{'japan': 1, 'kyoto': 2, 'osaka': 1, 'taiwan': 1}

**WORD COUNT FOR CLASS = NO**

In [ ]:

word_list_no = vec_no.get_feature_names();    
count_list_no = X_no.toarray().sum(axis=0) 
freq_no = dict(zip(word_list_no,count_list_no))
freq_no

{'london': 1, 'macao': 1, 'shanghai': 1, 'taipei': 1, 'taiwan': 2}

TOTAL WORDS IN DICTIONARY 

In [ ]:
dictionary = list(set(word_list_yes) | set(word_list_no))
dictionary,len(dictionary)

(['london',
  'shanghai',
  'macao',
  'taipei',
  'japan',
  'osaka',
  'taiwan',
  'kyoto'],
 8)

**PROBABILITIY OF WORDS BELONGING TO CLASS = YES**

In [ ]:
prob_yes = []
for word,count in zip(word_list_yes,count_list_yes):
  prob_yes.append((count+1)/(sum(count_list_yes)+len(dictionary)))
yes_dict = dict(zip(word_list_yes,prob_yes))

yes_dict

{'japan': 0.15384615384615385,
 'kyoto': 0.23076923076923078,
 'osaka': 0.15384615384615385,
 'taiwan': 0.15384615384615385}

In [ ]:
pro_yesno = []
difference = set(dictionary).symmetric_difference(set(word_list_yes))
yes_no = list(difference)

for word in zip(yes_no , count_list_yes):
  pro_yesno.append(1/(sum(count_list_yes)+len(dictionary)))
yesno_dic = dict(zip(yes_no,pro_yesno))
yesno_dic

{'london': 0.07692307692307693,
 'macao': 0.07692307692307693,
 'shanghai': 0.07692307692307693,
 'taipei': 0.07692307692307693}

In [ ]:
final_yes_probabilities=yes_dict.update(yesno_dic)
yes_dict

{'japan': 0.15384615384615385,
 'kyoto': 0.23076923076923078,
 'london': 0.07692307692307693,
 'macao': 0.07692307692307693,
 'osaka': 0.15384615384615385,
 'shanghai': 0.07692307692307693,
 'taipei': 0.07692307692307693,
 'taiwan': 0.15384615384615385}

**PROBABILITY OF WORDS BELONGING TO CLASS = NO**

In [ ]:
prob_no = []
for count in count_list_no:
  prob_no.append((count+1)/(sum(count_list_no)+len(dictionary)))
no_dict=dict(zip(word_list_no,prob_no))
no_dict

{'london': 0.14285714285714285,
 'macao': 0.14285714285714285,
 'shanghai': 0.14285714285714285,
 'taipei': 0.14285714285714285,
 'taiwan': 0.21428571428571427}

In [ ]:
pro_noyes = []
difference = set(dictionary).symmetric_difference(set(word_list_no))
no_yes = list(difference)

for word in zip(no_yes , count_list_no):
  pro_noyes.append(1/(sum(count_list_no)+len(dictionary)))

noyes_dic = dict(zip(no_yes,pro_noyes))
noyes_dic

{'japan': 0.07142857142857142,
 'kyoto': 0.07142857142857142,
 'osaka': 0.07142857142857142}

In [ ]:
final_no_probabilities=no_dict.update(noyes_dic)
no_dict

{'japan': 0.07142857142857142,
 'kyoto': 0.07142857142857142,
 'london': 0.14285714285714285,
 'macao': 0.14285714285714285,
 'osaka': 0.07142857142857142,
 'shanghai': 0.14285714285714285,
 'taipei': 0.14285714285714285,
 'taiwan': 0.21428571428571427}

**PREPARING THE TEST CASE - TAIWAN TAIWAN KYOTO**

In [ ]:
new_sentence = 'taiwan taiwan kyoto'
new_word_list = word_tokenize(new_sentence)
new_word_list

['taiwan', 'taiwan', 'kyoto']

**FINDING PROBABILITY THAT TEST CASE BELONGS TO CLASS = YES**

In [ ]:
final_probability_yes = class_yes
for j in range(len(new_word_list)):
  final_probability_yes = yes_dict[new_word_list[j]]*final_probability_yes

final_probability_yes

0.002184797451069641

**FINDING PROBABILITY THAT TEST CASE BELONGS TO CLASS = NO**

In [ ]:
final_probability_no = class_no
for r in range(len(new_word_list)):
  final_probability_no = no_dict[new_word_list[r]]*final_probability_no

final_probability_no

0.0019679300291545188

**FINAL RESULT WHERE EITHER OF THE CLASS = YES OR NO WINS**

In [ ]:
predicted_data=[];

if(final_probability_no > final_probability_yes):
  print("the test data - Taiwan Taiwan Kyoto belongs to class NO ")
  predicted_data.append(['taiwan taiwan kyoto','no'])
  predicted_data=pd.DataFrame(predicted_data)

else:
  print("the test data - Taiwan Taiwan Kyoto belongs to class YES ")
  predicted_data.append(['taiwan taiwan kyoto','yes'])
  predicted_data=pd.DataFrame(predicted_data)


predicted_data.columns=['TEST DATA','PREDICTED CLASS']
predicted_data

the test data - Taiwan Taiwan Kyoto belongs to class YES 


,TEST DATA,PREDICTED CLASS
0,taiwan taiwan kyoto,yes


**EVALUATION**

In [ ]:
#defining the actual data
test_data = [['taiwan taiwan kyoto','yes']]
true_data = pd.DataFrame(test_data)
true_data.columns = ['TEST DATA','TRUE CLASS']
true_data

,TEST DATA,TRUE CLASS
0,taiwan taiwan kyoto,yes


In [ ]:
df =pd.DataFrame()
true_positive = 0
false_positive = 0 
true_negative = 0 
false_negative = 0 
for i in range(len(true_data)):
  for j in range(len(predicted_data)):
   if((true_data['TEST DATA'][i] == predicted_data['TEST DATA'][j] )):
     if(true_data['TRUE CLASS'][i] == 'yes'):
       if(predicted_data['PREDICTED CLASS'][j] == 'yes'):
         true_positive = true_positive+1
       else:
         false_negative = false_negative + 1
     if(true_data['TRUE CLASS'][i] == 'no'):
       if(predicted_data['PREDICTED CLASS'][j] == 'yes'):
         false_positive = false_positive+1
       else:
         true_negative = true_negative + 1
      
true_positive,false_positive,false_negative,true_negative

(1, 0, 0, 0)

**CONFUSION MATRIX**

In [ ]:
confusion_matrix=[[true_positive,false_positive],[false_negative,true_negative]]
confusion_matrix = pd.DataFrame(confusion_matrix)
confusion_matrix.columns =['ACTUAL YES CLASS', 'ACTUAL NO CLASS']
confusion_matrix.index = ['PREDICTED TO BE IN YES CLASS', 'PREDICTED TO BE IN NO CLASS']

In [ ]:
confusion_matrix

,ACTUAL YES CLASS,ACTUAL NO CLASS
PREDICTED TO BE IN YES CLASS,1,0
PREDICTED TO BE IN NO CLASS,0,0


**PRECISION AND RECALL**

In [ ]:
precision=true_positive/(true_positive+false_positive)
recall = true_positive/(true_positive+false_negative)
F1_value = 2*precision*recall/(precision+recall)

print("THE PRECISION IS:"+str(precision)+"\nTHE RECALL IS :"
+str(recall)+"\nTHE F1 VALUE IS :"+str(F1_value))

THE PRECISION IS:1.0
THE RECALL IS :1.0
THE F1 VALUE IS :1.0
